# Поиск мусора по ключам

# NLTK

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import spacy
import re
import pymorphy3

from tqdm import tqdm

In [2]:
nltk.download('punkt') 
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Silm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Silm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
df = pd.read_excel('Выборка_Казань.xlsx', sheet_name = 0)

In [3]:
df['Текст'] = df['Текст'].fillna('нет текста')  #в столбце 'Текст' есть пустые строки NaN

In [4]:
df['Текст'] = df['Текст'].astype(str)

In [5]:
def preprocess(text, punctuation_marks, morph):
    text = re.sub(r"\d+", "", text, flags = re.UNICODE)

    for s in punctuation_marks:
        if s in text:
            text = text.replace(s, ' ')  #а-a, е-e, о-o, р-p, с-c, у-y, х-x

    for lat_lett, kir_lett in {"a": "а", "e": "е", "o": "о", "p": "р", "c": "с", "y": "у", "x": "х"}.items():
        text = text.replace(lat_lett.lower(), kir_lett)
    
    text = re.sub(r"\s+", " ", text).strip()
   
    tokens = word_tokenize(text.lower())
    
    preprocessed_text = []
    
    for token in tokens:
        lemma = morph.parse(token)[0].normal_form
        preprocessed_text.append(lemma)

    lemmatized_text = " ".join(preprocessed_text)
    return lemmatized_text

In [6]:
punctuation_marks = ['«', '»', '№', '!', ',', '(', ')', ':', ';', '-', '—', '?', '.', '..', '...', '\\', '|', '/']
morph = pymorphy3.MorphAnalyzer()

In [7]:
text_lemm = []

for i in tqdm(range(0, len(df['Текст']), 1)):
    text_lemm.append(preprocess(df['Текст'][i], punctuation_marks, morph))

df['Текст_lemm_nltk'] = text_lemm

100%|████████████████████████████████████████████████████████████████████████████| 11942/11942 [02:38<00:00, 75.37it/s]


In [9]:
df['Текст_lemm_nltk'][11062]

"срочный продажа возможно по семейный ипотека квартира свободный планировка в престижный дом м собственный терраса предлагать ваш внимание квартира в жилой комплекс бизнес класс `` sаvin fаmil '' расположить в современный центр казань в новый савиновский район преимущество квартира свободный планировка м свой терраса патио высота потолок м семейный ипотека благоустроенный закрытый территория видеонаблюдение в двор ландшафтный дизайн детский и спортивный площадка зона отдых консьерж сервис при вход в дом каминный зал до метр высота потолок в лобби подземный паркинг в шаговый доступность дворец единоборство ак барс аквапарк ривьера стадион казань арена татнефть арена торговый центр и фитнес комплекс детский сад и школа наслаждаться жизнь в престижный район город казань либо инвестировать для сдача в аренда"

## Проверка на доли

In [23]:
trash_word = ['доля', 'общедолевая', 'доля в праве', 'статус доля'] 
not_trash_word = ['не доля', 'нет долей', 'долей нет', 'без долей', 'детская доля', 'дет доля', 'доля ребенка', 'доли детей',
                  'без долей и обременений', 'без долей обременений', 'нет залогов и долей', 'обременений и долей', 'обременений долей',
                  'не состоит в доле', 'не в доле', 'долей обременений', 'долей и обременений', 'детей и долей', 'Запретов Долей', 'долгов долей',
                  'дети доля', 'выделения долей', 'детей/долей', 'Возможна продажа доли', 'в доль', 'не разделен на доли', 'доли двум детям',
                  'дети в доли', 'занижений долей', 'наследников долей', 'долей опеки', 'капиталов и долей', 'долей несовершеннолетних',
                  'несовершеннолетних доли', 'оплата долями', 'долями в течение', 'долей и маткапитала', 'никаких долей', 'прописанных и долей',
                  'арестов и долей', 'мат капиталов долей', 'долей никаких', 'долей ни каких', 'никаких долей', 'ни каких долей']

trash_word_lem = []
not_trash_word_lem = []

for w in trash_word:
    trash_word_lem.append(preprocess(w, punctuation_marks, morph))
    
for w in not_trash_word:
    not_trash_word_lem.append(preprocess(w, punctuation_marks, morph))

print(trash_word_lem)
print(not_trash_word_lem)

['доля', 'общедолевой', 'доля в право', 'статус доля']
['не доля', 'нет доля', 'доля нет', 'без доля', 'детский доля', 'деть доля', 'доля ребёнок', 'доля ребёнок', 'без доля и обременение', 'без доля обременение', 'нет залог и доля', 'обременение и доля', 'обременение доля', 'не состоять в доля', 'не в доля', 'доля обременение', 'доля и обременение', 'ребёнок и доля', 'запрет доля', 'долг доля', 'ребёнок доля', 'выделение доля', 'ребёнок доля', 'возможный продажа доля', 'в доля', 'не раздельный на доля', 'доля два ребёнок', 'ребёнок в доля', 'занижение доля', 'наследник доля', 'доля опека', 'капитал и доля', 'доля несовершеннолетний', 'несовершеннолетний доля', 'оплата доля', 'доля в течение', 'доля и маткапитал', 'никакой доля', 'прописать и доля', 'арест и доля', 'мат капитал доля', 'доля никакой', 'доля ни какой', 'никакой доля', 'ни какой доля']


In [24]:
def is_trash(text):
    for word in not_trash_word_lem:  # сразу метим как не мусор, если слово в not_trash_word
        if word in text:
            return False
    for word in trash_word_lem:
        if word in text:
            return True
    return False

In [25]:
df['Мусор_доли'] = df['Текст_lemm_nltk'].apply(is_trash)
dol_trash = df[df['Мусор_доли'] == True]
dol_trash

,ID,Мусор,dup,outlier,out_PCA,Цена,Цена_перв,LN(Цена),BC(Цена),Цена_перв.1,...,Unnamed: 63,ПОИСК,Текст,Ремонт,Общая площадь,Год постройки,Этаж/Этажность,Ссылка,Текст_lemm_nltk,Мусор_доли
81,82_avito,1,0,0,NaN,3549000,3 549 000 ₽,15.082176,4.877504,NaN,...,NaN,0.0,Отличная альтернатива гостинкам в общежитиях. ...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/kvartira-s...,отличный альтернатива гостинка в общежитие кир...,True
476,477_avito,1,0,0,NaN,10000000,10 000 000 ₽,16.118096,4.927790,NaN,...,NaN,2.0,Предлагаю Вашему вниманию Шикарную квартиру в ...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/dolya_v_4-...,предлагать ваш внимание шикарный квартира в го...,True
1214,1215_avito,1,0,0,NaN,5000000,5 000 000 ₽,15.424948,4.895274,NaN,...,NaN,0.0,Арт. 67763539 Внимание! 1/2 Доля в 2-х комн кв...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/dolya_v_2-...,арт внимание доля в х комн квартира в жк бизне...,True
1425,1426_avito,1,0,0,NaN,4490000,4 490 000 ₽,15.317363,4.889823,NaN,...,NaN,0.0,Продается 1/2 Доля В Праве в 2-комнатной кварт...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/dolya_v_2-...,продаваться доля в право в комнатный квартира ...,True
1469,1470_avito,1,0,0,NaN,3000000,3 000 000 ₽,14.914123,4.868350,NaN,...,NaN,0.0,"Продам 1/2 долю в двух комнатной квартире, тор...",евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/dolya_v_2-...,продать доля в два комнатный квартира торг дос...,True
2130,2131_avito,1,0,0,NaN,4800000,4 800 000 ₽,15.384126,4.893219,NaN,...,NaN,0.0,"В продаже уютная, упакованная студия по адресу...",дизайнерская,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/kvartira-s...,в продажа уютный упаковать студия по адрес ул ...,True
3040,3041_avito,0,0,0,0.0,3700000,3 700 000 ₽,15.123843,4.879728,NaN,...,NaN,0.0,Продам упакованную квартиру — студию в Ново — ...,косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/kvartira-s...,продать упаковать квартира студия в новый сави...,True
3390,3391_avito,1,0,0,0.0,3900000,3 900 000 ₽,15.176487,4.882512,NaN,...,NaN,0.0,Пpодaм 1/2 дoлю в квартире 1996 гoдa. Вторoй с...,косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/dolya_v_1-...,продать доля в квартира год второй собственник...,True
3692,3693_avito,0,0,0,0.0,5799000,5 799 000 ₽,15.573196,4.902600,NaN,...,NaN,0.0,"646694 Предлагаем вашему вниманию просторную, ...",косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,предлагать ваш внимание просторный светлый уют...,True
4467,4468_avito,1,0,0,NaN,4500000,4 500 000 ₽,15.319588,4.889937,NaN,...,NaN,0.0,Арт. 67270019 Предлагаю вашему вниманию кварти...,косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/dolya_v_3-...,арт предлагать ваш внимание квартира по улица ...,True


In [26]:
#Сохраняем в Excel
writer = pd.ExcelWriter('Dol_trash_kaz.xlsx')
dol_trash.to_excel(writer, 'Dol_trash')
writer._save()

C:\Users\Silm\AppData\Local\Temp\ipykernel_11504\2427646682.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  dol_trash.to_excel(writer, 'Dol_trash')


## Проверка на аукционы

In [9]:
trash_word = ['аукцион', 'через электронный аукцион', 'о несостоятельности', 'исполнительное производство', 'продажа на торгах', 'реализация на торгах',
              'продажа через торги', 'реализация через торги', 'имущество должника', 'электронные торги', 'реализуется по поручению'] 
not_trash_word = ['не имеет исполнительных производств', 'нет исполнительных производств', 'без исполнительных производств',
                  'исполнительных производств нет']

trash_word_lem = []
not_trash_word_lem = []

for w in trash_word:
    trash_word_lem.append(preprocess(w, punctuation_marks, morph))
    
for w in not_trash_word:
    not_trash_word_lem.append(preprocess(w, punctuation_marks, morph))

print(trash_word_lem)
print(not_trash_word_lem)

['аукцион', 'через электронный аукцион', 'о несостоятельность', 'исполнительный производство', 'продажа на торг', 'реализация на торг', 'продажа через торг', 'реализация через торг', 'имущество должник', 'электронный торг', 'реализоваться по поручение']
['не иметь исполнительный производство', 'нет исполнительный производство', 'без исполнительный производство', 'исполнительный производство нет']


In [10]:
def is_trash(text):
    for word in not_trash_word_lem:  
        if word in text:
            return False
    for word in trash_word_lem:
        if word in text:
            return True
    return False

In [13]:
df['Мусор_аук'] = df['Текст_lemm_nltk'].apply(is_trash)
auk_trash = df[df['Мусор_аук'] == True]
auk_trash

,ID,Мусор,dup,outlier,out_PCA,Цена,Цена_перв,LN(Цена),BC(Цена),Цена_перв.1,...,Unnamed: 63,ПОИСК,Текст,Ремонт,Общая площадь,Год постройки,Этаж/Этажность,Ссылка,Текст_lemm_nltk,Мусор_аук
4934,4935_avito,1,0,0,NaN,10000000,от 10 000 000 ₽,16.118096,4.927790,NaN,...,NaN,0.0,"Квартира, общая площадь 54.9 кв. М., этаж 9, к...",косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/auktsion_3...,квартира общий площадь кв м этаж кадастровый н...,True
5781,5884_avito,1,0,0,NaN,7000000,от 7 000 000 ₽,15.761421,4.911604,NaN,...,NaN,0.0,"Квартира, расположенная по адресу: Республика ...",косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/auktsion_3...,квартира расположить по адрес республика татар...,True
5931,6049_avito,1,0,0,NaN,10457483,от 10 457 483 ₽,16.162828,4.929742,NaN,...,NaN,0.0,"Продается трехкомнатная квартира, расположенна...",косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/auktsion_3...,продаваться трехкомнатный квартира расположить...,True
6626,6774_avito,1,0,0,NaN,1602400,от 1 602 400 ₽,14.287013,4.831433,NaN,...,NaN,0.0,В процедуре исполнительного производства с тор...,требует_ремонта,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/auktsion_1...,в процедура исполнительный производство с торг...,True
6961,7109_avito,1,0,0,NaN,7200000,от 7 200 000 ₽,15.789592,4.912924,NaN,...,NaN,0.0,Hа электpoнныx тopгах Дешеbле Pынкa прoдaется:...,требует_ремонта,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/auktsion_2...,hа электронный торг дешеbль pынок продаваться ...,True
8222,9176_avito,1,0,0,NaN,4382982,от 4 382 982 ₽,15.293240,4.888585,NaN,...,NaN,0.0,В процедуре исполнительного производства с тор...,требует_ремонта,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/auktsion_2...,в процедура исполнительный производство с торг...,True


In [14]:
#Сохраняем в Excel
writer = pd.ExcelWriter('Auk_trash_kazan.xlsx')
auk_trash.to_excel(writer, 'Auk_trash')
writer._save()

C:\Users\Silm\AppData\Local\Temp\ipykernel_5748\1204865606.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  auk_trash.to_excel(writer, 'Auk_trash')


## Проверка на вид объекта (исключение коттеджей, таунхаусов, жилых домов и пр.)

In [84]:
trash_word = ['таунхаус', 'индивидуальный жилой дом', 'доля дома', 'доля жилого дома', 'квартира со своим участком', 'квартира с земельным участком',
              'с земельным участком', 'свой земельный участок', 'статус нежилое', 'под нежилое', 'продам нежилое помещение', 'по документам нежилое',
             'земля в собственности'] 
not_trash_word = [ 'обмен на земельный участок', 'обмен жилой (квартиры, комнаты, индивидуальный жилой дом, земельные участки, таунхаусы, коттеджи)',
                  'квартиру или дом, таунхаус', 'На первом этаже нежилое помещение', 'перевести в нежилое', 'жилой дом, земельные участки, таунхаусы, коттеджи',
                 'Над этажом есть нежилое помещение', 'Идеально под нежилое', 'Подходит для перевода под нежилое помещение', 'можно перевести в нежилое',
                  'дом, коттедж, дача, снт, таунхаус, земельный участок,', 'Можно переделать квартиру под нежилое помещение', 'вариант для оформления на нежилое помещение',
                 'нежилое помещение над квартирой', 'подходит под нежилое', 'детский доля дом', 'дет. Долей. Дoм', 'Первый этаж полностью занимают нежилые',
                 'с нежилыми помещениями, в том числе', 'вид на лесной массив на горизонте и индивидуальные жилые дома', 'обмен на дом или таунхаус', 
                 'Обмен на автомобиль, квартиру или таунхаус', 'обмен на таунхаус']

trash_word_lem = []
not_trash_word_lem = []

for w in trash_word:
    trash_word_lem.append(preprocess(w, punctuation_marks, morph))
    
for w in not_trash_word:
    not_trash_word_lem.append(preprocess(w, punctuation_marks, morph))

print(trash_word_lem)
print(not_trash_word_lem)

['таунхаус', 'индивидуальный жилой дом', 'доля дом', 'доля жилой дом', 'квартира с свой участок', 'квартира с земельный участок', 'с земельный участок', 'свой земельный участок', 'статус нежилой', 'под нежилой', 'продать нежилой помещение', 'по документ нежилой', 'земля в собственность']
['обмен на земельный участок', 'обмен жилой квартира комната индивидуальный жилой дом земельный участок таунхаус коттедж', 'квартира или дом таунхаус', 'на первый этаж нежилой помещение', 'перевести в нежилой', 'жилой дом земельный участок таунхаус коттедж', 'над этаж есть нежилой помещение', 'идеально под нежилой', 'подходить для перевод под нежилой помещение', 'можно перевести в нежилой', 'дом коттедж дача снт таунхаус земельный участок', 'можно переделать квартира под нежилой помещение', 'вариант для оформление на нежилой помещение', 'нежилой помещение над квартира', 'подходить под нежилой', 'детский доля дом', 'деть доля дом', 'первый этаж полностью занимать нежилой', 'с нежилой помещение в тот чис

In [85]:
def is_trash(text):
    for word in not_trash_word_lem:  
        if word in text:
            return False
    for word in trash_word_lem:
        if word in text:
            return True
    return False

In [86]:
df['Мусор_тип'] = df['Текст_lemm_nltk'].apply(is_trash)
type_trash = df[df['Мусор_тип'] == True]
type_trash

,ID,Мусор,dup,outlier,out_PCA,Цена,Цена_перв,LN(Цена),BC(Цена),Цена_перв.1,...,Unnamed: 63,ПОИСК,Текст,Ремонт,Общая площадь,Год постройки,Этаж/Этажность,Ссылка,Текст_lemm_nltk,Мусор_тип
4245,4246_avito,0,0,0,0.0,7500000,7 500 000 ₽,15.830414,4.914823,NaN,...,NaN,0.0,Продаётся 2 комнатная квартира с земельным уча...,косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,продаваться комнатный квартира с земельный уча...,True
4558,4559_avito,1,0,0,NaN,3999000,3 999 000 ₽,15.201555,4.883827,NaN,...,NaN,0.0,Уважаемые покупатели! Предлагаем Вашему вниман...,косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/3-k._kvart...,уважаемый покупатель предлагать ваш внимание х...,True
6087,6235_avito,1,0,0,NaN,7908920,7 908 920 ₽,15.883502,4.917271,NaN,...,NaN,0.0,Таунхаусы расположен в селе Сокуры Лаишевского...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,таунхаус расположить в село сокура лаишевский ...,True
6116,6264_avito,1,1,0,NaN,7908920,7 908 920 ₽,15.883502,4.917271,NaN,...,NaN,0.0,Таунхаусы расположен в селе Сокуры Лаишевского...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,таунхаус расположить в село сокура лаишевский ...,True
6126,6274_avito,1,1,0,NaN,7908920,7 908 920 ₽,15.883502,4.917271,NaN,...,NaN,0.0,Таунхаусы расположен в селе Сокуры Лаишевского...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,таунхаус расположить в село сокура лаишевский ...,True
6133,6281_avito,1,1,0,NaN,7908920,7 908 920 ₽,15.883502,4.917271,NaN,...,NaN,0.0,Таунхаусы расположен в селе Сокуры Лаишевского...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,таунхаус расположить в село сокура лаишевский ...,True
6158,6306_avito,1,1,0,NaN,7908920,7 908 920 ₽,15.883502,4.917271,NaN,...,NaN,0.0,Таунхаусы расположен в селе Сокуры Лаишевского...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,таунхаус расположить в село сокура лаишевский ...,True
6170,6318_avito,1,1,0,NaN,7908920,7 908 920 ₽,15.883502,4.917271,NaN,...,NaN,0.0,Таунхаусы расположен в селе Сокуры Лаишевского...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,таунхаус расположить в село сокура лаишевский ...,True
6203,6351_avito,1,0,0,NaN,13200000,13 200 000 ₽,16.395727,4.939636,NaN,...,NaN,0.0,Продается Шикарный 2Х Этажный Таунхаус 136 (14...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,продаваться шикарный х этажный таунхаус кв м в...,True
6348,6496_avito,1,0,0,NaN,4814497,4 814 497 ₽,15.387142,4.893371,NaN,...,NaN,0.0,Таунхаусы расположен в селе Сокуры Лаишевского...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,таунхаус расположить в село сокура лаишевский ...,True


In [87]:
#Сохраняем в Excel
writer = pd.ExcelWriter('type_trash_kazan.xlsx')
type_trash.to_excel(writer, 'type_trash')
writer._save()

C:\Users\Silm\AppData\Local\Temp\ipykernel_7396\1998625131.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  type_trash.to_excel(writer, 'type_trash')


## Проверка на вид объекта (комнаты, общежития)

In [32]:
trash_word = ['комната в общежитии', 'продам комнату', 'продажа комнаты', 'по документам комната', 'ЕГРН комната', 'статус комната', 'в общежитии',
              'комната в коммунальной', 'коммунальная квартира', 'без санузла'] 
not_trash_word = ['Не бывшее общежитие,', 'Продам 1 комнат. Квартиру']

trash_word_lem = []
not_trash_word_lem = []

for w in trash_word:
    trash_word_lem.append(preprocess(w, punctuation_marks, morph))
    
for w in not_trash_word:
    not_trash_word_lem.append(preprocess(w, punctuation_marks, morph))

print(trash_word_lem)
print(not_trash_word_lem)

['комната в общежитие', 'продать комната', 'продажа комната', 'по документ комната', 'егрн комната', 'статус комната', 'в общежитие', 'комната в коммунальный', 'коммунальный квартира', 'без санузел']
['не бывший общежитие', 'продать комната квартира']


In [33]:
def is_trash(text):
    for word in not_trash_word_lem:  
        if word in text:
            return False
    for word in trash_word_lem:
        if word in text:
            return True
    return False

In [34]:
df['Мусор_комната'] = df['Текст_lemm_nltk'].apply(is_trash)
room_trash = df[df['Мусор_комната'] == True]
room_trash

,ID,Мусор,dup,outlier,out_PCA,Цена,Цена_перв,LN(Цена),BC(Цена),Цена_перв.1,...,Unnamed: 63,ПОИСК,Текст,Ремонт,Общая площадь,Год постройки,Этаж/Этажность,Ссылка,Текст_lemm_nltk,Мусор_комната
62,63_avito,0,0,0,0.0,3150000,3 150 000 ₽,14.962913,4.871039,NaN,...,NaN,0.0,В Продаже Комната ( Статус Квартира) В Кирпичн...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,в продажа комната статус квартира в кирпичный ...,True
81,82_avito,1,0,0,NaN,3549000,3 549 000 ₽,15.082176,4.877504,NaN,...,NaN,0.0,Отличная альтернатива гостинкам в общежитиях. ...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/kvartira-s...,отличный альтернатива гостинка в общежитие кир...,True
103,104_avito,0,0,0,0.0,2750000,2 750 000 ₽,14.827111,4.863493,NaN,...,NaN,0.0,В прoдaжe cтудия в нoвом ЖК Советскoго pайoна!...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/kvartira-s...,в продажа студия в новый жк советский район об...,True
214,215_avito,0,0,0,0.0,3350000,3 350 000 ₽,15.024471,4.874394,NaN,...,NaN,0.0,Продаётся светлая 1комнатная квартира с отличн...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/kvartira-s...,продаваться светлый комнатный квартира с отлич...,True
242,243_avito,0,0,0,0.0,3300000,3 200 000 ₽,15.009433,4.871901,NaN,...,NaN,0.0,Квартира от собственника! Только наличный расч...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/kvartira-s...,квартира от собственник только наличный расчёт...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11287,2891_этажи,1,0,0,NaN,1800000,1 800 000,14.403297,4.838622,1900000.0,...,NaN,0.0,Отличное предложение ! \nПродается комната в к...,косметический ремонт,7 м²,1966,5 из 5,https://kazan.etagi.com/realty/8585395/,отличный предложение продаваться комната в ком...,True
11321,2925_этажи,1,0,0,NaN,1200000,1 200 000,13.997832,4.812835,NaN,...,NaN,0.0,Продаётся комната в коммунальной трёхкомнатной...,косметический ремонт,10 м²,1948,2 из 3,https://kazan.etagi.com/realty/8405128/,продаваться комната в коммунальный трёхкомнатн...,True
11323,2927_этажи,0,0,0,0.0,2250000,2 250 000,14.626441,4.851971,NaN,...,NaN,0.0,Продаю шикарную гостинку со статусом комнаты. ...,косметический ремонт,13.4 м²,1984,9 из 9,https://kazan.etagi.com/realty/8390761/,продавать шикарный гостинка с статус комната в...,True
11324,2928_этажи,0,0,0,0.0,2990000,2 990 000,14.910784,4.868166,NaN,...,NaN,0.0,Продаю шикарную гостинку со статусом комнаты. ...,косметический ремонт,17.4 м²,1959,1 из 5,https://kazan.etagi.com/realty/8390748/,продавать шикарный гостинка с статус комната в...,True


In [23]:
#Сохраняем в Excel
writer = pd.ExcelWriter('room_trash_kazan.xlsx')
room_trash.to_excel(writer, 'room_trash')
writer._save()

C:\Users\Silm\AppData\Local\Temp\ipykernel_9684\3938254791.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  room_trash.to_excel(writer, 'room_trash')


## Проверка на ввод дома (строящиеся ЖК)

In [101]:
trash_word = ['строящемся доме', 'строящемся ЖК', 'строящемся жилом комплексе', 'планируется досрочная сдача', 'сдача раньше срока',
             'сдать раньше срока', 'планируемый срок', 'сдача через', 'будет сдан', 'сдается в году', 'сдается в квартале', 'сдача год',
             'сдача квартал', 'сдача начало', 'сдача конец', 'сдача лето', 'сдача зима', 'сдача осень', 'сдача весна', 'вводится в эксплуатацию в',
              'в новом строящемся','сдача год', 'сдача квартал', 'окончания строительства', 'передача ключей в', 'передача ключей год', 'дом не сдан', 
              'сдача планируется', 
              'сдается в предчистовой', 'передача ключей квартал', 'передача ключей лето', 'передача ключей зима', 'передача ключей весна',
              'передача ключей осень', 'передача ключей начало', 'передача ключей конец', 'срок выдачи ключей',  
             'сдается в черновой', 'будет сдана в'] 
not_trash_word = ['дом сдан', 'метро срок сдачи', 'школа сдача', 'рядом строится жк', 'школа, которая будет сдаваться в', 'рядом в 2 минутах строятся жилые комплексы',
                 'Дом был сдан', 'планируется рост цен на недвижимость, по окончанию строительства новых ЖК', 'Дом года', 'строиться дома не будут', 'Дом был сдан',
                  'был сдан', 'интересный локация для сдача', 'передача ключей в день', 'построенном жилом дoмe', 'передача ключ в дата полный расчёт',
                  'года постройки', 'в сданном доме', 'передача ключей, встречный вариант подобран', 'рядом строятся', 'при окончании строительства всех корпусов',
                 'медучреждение( сдача конец', 'Дом уже сдан', 'сданном доме', 'Дальше строятся дома', 'от строящихся домов открылся', 
                  'Передача ключей возможна в день', 'В собственности одной семьи с момента окончания строительства', 'был заселен', 'дет сад сдача',
                 'ключи переданы', 'не будут строиться дома', 'Дом построен', 'детский сад(сдача', 'По соседству строится ЖК', 'школа, которая сдается',
                 'школа строится в соседнем дворе (сдача', 'Документы получены', 'для проживания так и для сдачи', 'школа во дворе, сдача',
                  'ниже аналогов в строящихся', 'отсутствием бесконечных строящихся жилых', 'построенные и строящиеся ЖК', 'школа строиться через 2 дома сдача',
                 'строится школа (срок сдачи', 'Напротив строится жилой комплекс', 'школа (вводится в эксплуатацию']

trash_word_lem = []
not_trash_word_lem = []

for w in trash_word:
    trash_word_lem.append(preprocess(w, punctuation_marks, morph))
    
for w in not_trash_word:
    not_trash_word_lem.append(preprocess(w, punctuation_marks, morph))

print(trash_word_lem)
print(not_trash_word_lem)

['строиться дом', 'строиться жк', 'строиться жилой комплекс', 'планироваться досрочный сдача', 'сдача ранний срок', 'сдать ранний срок', 'планировать срок', 'сдача через', 'быть сдать', 'сдаваться в год', 'сдаваться в квартал', 'сдача год', 'сдача квартал', 'сдача начало', 'сдача конец', 'сдача лето', 'сдача зима', 'сдача осень', 'сдача весна', 'вводиться в эксплуатация в', 'в новый строиться', 'сдача год', 'сдача квартал', 'окончание строительство', 'передача ключ в', 'передача ключ год', 'дом не сдать', 'сдача планироваться', 'сдаваться в предчистовой', 'передача ключ квартал', 'передача ключ лето', 'передача ключ зима', 'передача ключ весна', 'передача ключ осень', 'передача ключ начало', 'передача ключ конец', 'срок выдача ключ', 'сдаваться в черновой', 'быть сдать в']
['дом сдать', 'метро срок сдача', 'школа сдача', 'рядом строиться жк', 'школа который быть сдаваться в', 'рядом в минута строиться жилой комплекс', 'дом быть сдать', 'планироваться рост цена на недвижимость по оконча

In [102]:
def is_trash(text):
    for word in not_trash_word_lem:  
        if word in text:
            return False
    for word in trash_word_lem:
        if word in text:
            return True
    return False

In [103]:
df['Мусор_не_сдан'] = df['Текст_lemm_nltk'].apply(is_trash)
yb_trash = df[df['Мусор_не_сдан'] == True]
yb_trash

,ID,Мусор,dup,outlier,out_PCA,Цена,Цена_перв,LN(Цена),BC(Цена),Цена_перв.1,...,ПОИСК,Текст,Ремонт,Общая площадь,Год постройки,Этаж/Этажность,Ссылка,Текст_lemm_nltk,Мусор_комната,Мусор_не_сдан
2040,2041_avito,1,0,0,NaN,12650000,12 650 000 ₽,16.353168,4.937861,NaN,...,0.0,Продается квартира в ЖК «Savin Family» на 4 эт...,без_отделки,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,продаваться квартира в жк sаvin fаmil на этаж ...,False,True
2054,2055_avito,0,0,0,0.0,8300000,8 300 000 ₽,15.931766,4.919474,NaN,...,0.0,Новая 2-к квартира в ЖК Светлая Долина ПК-3-2 ...,без_отделки,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,новый к квартира в жк светлый долина пк от соб...,False,True
3965,3966_avito,1,0,0,NaN,7500000,7 500 000 ₽,15.830414,4.914823,NaN,...,0.0,"Арт. 66062199 Казань, Советский район, ул. Хал...",косметический,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,арт казань советский район ул халитовый предла...,False,True
6051,6199_avito,0,0,0,0.0,10750000,10 750 000 ₽,16.190416,4.930937,NaN,...,0.0,Название новостройки: ЖК «Родина». Адрес: г. К...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,название новостройка жк родина адрес г казань ...,False,True
6182,6330_avito,1,0,0,NaN,16500000,16 500 000 ₽,16.618871,4.948705,NaN,...,0.0,Продается евротрех комнатная квартира в ЖК «Sa...,предчистовая,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,продаваться евротреха комнатный квартира в жк ...,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11184,2788_этажи,0,0,1,0.0,4500000,4 500 000,15.319588,4.889937,4999000.0,...,0.0,ВЫ ВСЕ ЕЩЕ ПЛАТИТЕ ЗА ЧУЖУЮ ИПОТЕКУ? \nА хотит...,косметический ремонт,58.8 м²,2019,2 из 3,https://kazan.etagi.com/realty/9045987/,вы всё ещё платить за чужой ипотека а хотеть с...,False,True
11332,2936_этажи,0,0,0,0.0,3750000,3 750 000,15.137266,4.880440,NaN,...,0.0,Представляем вашему вниманию отличную светлую ...,улучшенная черновая отделка,27.9 м²,2023,7 из 9,https://kazan.etagi.com/realty/8382384/,представлять ваш внимание отличный светлый сту...,False,True
11406,3010_этажи,0,0,0,0.0,6050000,6 050 000,15.615569,4.904656,NaN,...,0.0,"Уважаемые покупатели, продается 1 комнатная кв...",улучшенная черновая отделка,37.1 м²,2022,13 из 25,https://kazan.etagi.com/realty/7778398/,уважаемый покупатель продаваться комнатный ква...,False,True
11425,3029_этажи,0,0,0,0.0,4600000,4 600 000,15.341567,4.891059,4700000.0,...,0.0,ВНИМАНИЕ! ПЕРЕУСТУПКА! Ключи в сентябре!\nПрод...,улучшенная черновая отделка,33.95 м²,2023,9 из 20,https://kazan.etagi.com/realty/7474188/,внимание переуступка ключ в сентябрь продавать...,False,True


In [80]:
#Сохраняем в Excel
writer = pd.ExcelWriter('yb_trash_kazan.xlsx')
yb_trash.to_excel(writer, 'yb_trash')
writer._save()

C:\Users\Silm\AppData\Local\Temp\ipykernel_9684\876891515.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  yb_trash.to_excel(writer, 'yb_trash')


## Проверка на этаж (цоколи)

In [132]:
trash_word = ['в жилом цоколе', 'квартира в цоколе', 'студия в цоколе', 'расположена в цоколе', 'квартира на цокольном этаже',
              'студия на цокольном этаже', 'по документам цокольный', 'по документам цоколь', 'сололифт', 
              'продам помещение в цоколе', 'апартаменты в цокольном', 'апартамент в цоколе', 'этаж цокольный'] 
not_trash_word = ['на цокольном этаже имеется', 'наличии ЖИЛОГО цокольного', 'Под квартирой в цокольном этаже', 
                  'кладовая 8 кв.м. для каждой квартиры в цокольном этаже', '1-й этаж цокольный']

trash_word_lem = []
not_trash_word_lem = []

for w in trash_word:
    trash_word_lem.append(preprocess(w, punctuation_marks, morph))
    
for w in not_trash_word:
    not_trash_word_lem.append(preprocess(w, punctuation_marks, morph))

print(trash_word_lem)
print(not_trash_word_lem)

['в жилой цоколь', 'квартира в цоколь', 'студия в цоколь', 'расположить в цоколь', 'квартира на цокольный этаж', 'студия на цокольный этаж', 'по документ цокольный', 'по документ цоколь', 'сололифт', 'продать помещение в цоколь', 'апартамент в цокольный', 'апартамент в цоколь', 'этаж цокольный']
['на цокольный этаж иметься', 'наличие жилой цокольный', 'под квартира в цокольный этаж', 'кладовая кв м для каждый квартира в цокольный этаж', 'й этаж цокольный']


In [133]:
def is_trash(text):
    for word in not_trash_word_lem:  
        if word in text:
            return False
    for word in trash_word_lem:
        if word in text:
            return True
    return False

In [134]:
df['Мусор_цоколь'] = df['Текст_lemm_nltk'].apply(is_trash)
ground_trash = df[df['Мусор_цоколь'] == True]
ground_trash

,ID,Мусор,dup,outlier,out_PCA,Цена,Цена_перв,LN(Цена),BC(Цена),Цена_перв.1,...,Текст,Ремонт,Общая площадь,Год постройки,Этаж/Этажность,Ссылка,Текст_lemm_nltk,Мусор_комната,Мусор_не_сдан,Мусор_цоколь
6553,6701_avito,1,0,0,NaN,5500000,5 500 000 ₽,15.520259,4.900008,NaN,...,"Продается двухкомнатная кв 43,4 кв. М. На цоко...",требует_ремонта,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,продаваться двухкомнатный кв кв м на цокольный...,False,False,True


In [124]:
#Сохраняем в Excel
writer = pd.ExcelWriter('ground_trash_kazan.xlsx')
ground_trash.to_excel(writer, 'ground_trash')
writer._save()

C:\Users\Silm\AppData\Local\Temp\ipykernel_9684\2846458687.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  ground_trash.to_excel(writer, 'ground_trash')


## Проверка: наличие террасы

In [40]:
trash_word = ['с террасой', 'терраса кв', 'терраса м', 'две террасы', '+ терраса', ' терраса общей площадью',
              'собственная терраса', 'открытая терраса', 'терраса квадратов', 'своя терраса', 'видовая терраса',
              'выходом на террасу', 'с двумя большими террасами', 'есть большая терраса', 'своя летняя терраса', 
              'огороженная терраса', 'терраса площадью', 'терраса с видом', 'выходом на просторную террасу', 'площадь терраса',
              'закрытая терраса', 'терраса метров', 'большая терраса', 'огромная терраса', 'терраса более м', 
              'с просторными террасами']  
not_trash_word = ['террасой или', 'с террасой на первых', 'формат квартир с террасой', 'общая терраса', 'общая открытая терраса',
                  'организовать собственную террасу', 'сделать собственную террасу', 'у каждого дома открытые террасы', 
                  'доступны квартиры с собственной террасой', 'как небольшая терраса', 'дом с террасой', 'террасы квартир первого',
                 'террасы на этажах']


trash_word_lem = []
not_trash_word_lem = []

for w in trash_word:
    trash_word_lem.append(preprocess(w, punctuation_marks, morph))
    
for w in not_trash_word:
    not_trash_word_lem.append(preprocess(w, punctuation_marks, morph))

print(trash_word_lem)
print(not_trash_word_lem)

['с терраса', 'терраса кв', 'терраса м', 'два терраса', '+ терраса', 'терраса общий площадь', 'собственный терраса', 'открытый терраса', 'терраса квадрат', 'свой терраса', 'видовой терраса', 'выход на терраса', 'с два больший терраса', 'есть больший терраса', 'свой летний терраса', 'огородить терраса', 'терраса площадь', 'терраса с вид', 'выход на просторный терраса', 'площадь терраса', 'закрытый терраса', 'терраса метр', 'больший терраса', 'огромный терраса', 'терраса более м', 'с просторный терраса']
['терраса или', 'с терраса на первый', 'формат квартира с терраса', 'общий терраса', 'общий открытый терраса', 'организовать собственный терраса', 'сделать собственный терраса', 'у каждый дом открытый терраса', 'доступный квартира с собственный терраса', 'как небольшой терраса', 'дом с терраса', 'терраса квартира первый', 'терраса на этаж']


In [41]:
def is_trash(text):
    for word in not_trash_word_lem:  
        if word in text:
            return False
    for word in trash_word_lem:
        if word in text:
            return True
    return False

In [42]:
df['Map_terrace'] = df['Текст_lemm_nltk'].apply(is_trash)
map_terrace = df[df['Map_terrace'] == True]
map_terrace

,ID,Мусор,dup,outlier,out_PCA,Цена,Цена_перв,LN(Цена),BC(Цена),Цена_перв.1,...,ПОИСК,Текст,Ремонт,Общая площадь,Год постройки,Этаж/Этажность,Ссылка,Текст_lemm_nltk,Map_floor,Map_terrace
1009,1010_avito,0,0,0,0.0,10500000,10 500 000 ₽,16.166886,4.929918,NaN,...,0.0,"Пpoдaется однокомнатная светлая, уютная кварти...",евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,продаваться однокомнатный светлый уютный кварт...,False,True
1671,1672_avito,0,0,0,0.0,14250000,14 250 000 ₽,16.472267,4.942791,NaN,...,0.0,Продается упакованная и готовая квартира в ЖК ...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,продаваться упаковать и готовый квартира в жк ...,False,True
1790,1791_avito,0,0,0,0.0,15500000,15 500 000 ₽,16.556351,4.946203,NaN,...,0.0,Продам 2-комнатную квартиру в ЖК «Ричмонд». ЖК...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,продать комнатный квартира в жк ричмонд жк рич...,False,True
1849,1850_avito,0,0,0,0.0,14900000,14 900 000 ₽,16.516872,4.944608,NaN,...,0.0,Просторная двухкомнатная квартира- распашонка ...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,просторный двухкомнатный квартира распашонка м...,False,True
2065,2066_avito,0,0,0,1.0,25900000,25 900 000 ₽,17.069754,4.965874,NaN,...,0.0,Расположение в самой красивой и тихой части це...,дизайнерская,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/3-k._kvart...,расположение в сам красивый и тихий часть цент...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11223,2827_этажи,0,0,0,1.0,19000000,23 000 000,16.759950,4.954239,23500000.0,...,1.0,В продаже ЭКСКЛЮЗИВНЫЙ Шикарный пентхаус в жил...,улучшенная черновая отделка,199.7 м²,2016,19 из 19,https://kazan.etagi.com/realty/8879246/,в продажа эксклюзивный шикарный пентхаус в жил...,False,True
11224,2828_этажи,0,0,1,NaN,90000000,90 000 000,18.315320,5.006191,NaN,...,0.0,ПРОДАЕТСЯ САМАЯ БОЛЬШАЯ КВАРТИРА В КАЗАНИ!\n\n...,современный ремонт,316.9 м²,2003,4 из 4,https://kazan.etagi.com/realty/8868700/,продаваться самый больший квартира в казань не...,True,True
11233,2837_этажи,0,0,0,0.0,49500000,49 500 000,17.717483,4.988052,NaN,...,0.0,Читайте описание!\nВидовая квартира в новом ЖК...,улучшенная черновая отделка,180.6 м²,2021,23 из 25,https://kazan.etagi.com/realty/8834079/,читать описание видовой квартира в новый жк at...,False,True
11337,2941_этажи,0,0,0,0.0,14000000,14 000 000,16.454568,4.942065,NaN,...,0.0,Продается 2 комнатная квартира в жилом комплек...,улучшенная черновая отделка,71.1 м²,2023,18 из 26,https://kazan.etagi.com/realty/8360895/,продаваться комнатный квартира в жилой комплек...,False,True


In [43]:
#Сохраняем в Excel
writer = pd.ExcelWriter('terrace_kzn_1.xlsx')
map_terrace.to_excel(writer, 'map_terrace')
writer._save()

C:\Users\Silm\AppData\Local\Temp\ipykernel_9192\944116028.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  map_terrace.to_excel(writer, 'map_terrace')


## Проверка: многоуровневость

In [51]:
trash_word = ['двухуровневая квартира', 'уровневая квартира', 'Квартира двухуровневая', 'многоуровневая квартира', 'находящаяся на втором уровне',
              'уровневая квартира', 'двухэтажная квартира', 'квартира двухэтажная', 'Двухуровневый пентхаус', 'пентхаус двухуровневый', 'первом уровне квартиры', 
              'Квартира трехуровневая', 'трехуровневая Квартира', 'двухуровневая однокомнатная квартира', 'двухуровневая к квартира', 'уровневая к квартира',
              'двухуровневая комнатная квартира', 'расположилась на двух уровнях', 'двухуровневая трешка', 'квартира в двух уровнях', 
              'двухуровневая x комнатная квартира','уровневую х комнатную квартиру', 'Продается двухуровневая', 'продам двухуровневую',
              'уровневый пентхаус', 'м в двух уровнях', 'двухуровневой планировкой', 'этажная квартира', 'пентхаус в двух уровнях',
              'Продается х уровневая', 'Квартира находится в уровнях','Квартира в уровнях', 'состоит из двух уровней', 'уровненный пентхаус', 
              'Квартира х уровневая', 'двухъярусная квартира', 'Доступ в квартиру с 12 и с 13 этажа', 'Двухэтажный пентхаус', 
              'Квартира уникальная двухуровневая', 'В продаже ДВУХУРОВНЕВАЯ', 'двухуровневая трехкомнатная  квартира', 'х этажный Пентхаус',
             'Продам двух-уровневую', 'В квартире два этажа', 'двухуровневая евро трёшка', 'Квартира уникальная двухуровневая',
             'В продаже просторная двухуровневая', 'Квартира занимает 1 и 2 этаж', 'Двухуровневая мансардная квартира',
              'на первом уровне + на втором уровне', 'На первом уровне расположены', 'Двухуровневая, просторная квартира', 'планировка двухуровневая',
             'уровневую 4-комнатную квартиру', 'Квартира находится на 21 и 22 этажах', 'Уникальная двухуровневая евро', 'Квартира находится на 5-м и 6-м этажах',
             'Квартира в 2-х уровнях', 'Пентхаус двухэтажный', 'Пентхаус в три уровня', 'Пентхаус расположен в двух уровнях']

not_trash_word = ['создания двухуровневой квартиры', 'возможностью устройства двухуровневой квартиры', 'Двухуровневый подземный паркинг',
                  'наземная многоуровневая', 'двухуровневые подвесные потолки', 'дом кирпичный ти этажный', 'позволяет сделать двухуровневую квартиру',
                 'кирпичный этажный квартира', 'Двухъярусные квартиры. Квартиры оснащенные террасами', 'малоэтажный квартира',
                 'высокоэтажный квартира', 'площадка находится на втором уровне дома', 'секция 10-ти этажная', 
                  'возможность сделать двухуровневую квартиру', 'двухуровневый и натяжной потолки', 'Дом 26-ти этажный', 
                 'сделать 2-х уровневую квартиру', 'дом 83 серии, 9-ти этажный', 'сделать двухуровневую квартиру', 'стилобат, состоящий из двух уровней',
                 'одноуровневый пентхаус', 'Дом 6 этажный, квартира', 'Дом монолитно-кирпичный 16ти этажный. Квартира', 'Дом трехэтажный, 18 квартир',
                 'реализации проекта двухуровневой квартиры', 'Наш литер — 9-ти этажный. Квартира', 'позволяет сделать ее полноценной, двухэтажной квартирой',
                 'На первом уровне расположены столы']


trash_word_lem = []
not_trash_word_lem = []

for w in trash_word:
    trash_word_lem.append(preprocess(w, punctuation_marks, morph))
    
for w in not_trash_word:
    not_trash_word_lem.append(preprocess(w, punctuation_marks, morph))

print(trash_word_lem)
print(not_trash_word_lem)


['двухуровневый квартира', 'уровневый квартира', 'квартира двухуровневый', 'многоуровневый квартира', 'находиться на второй уровень', 'уровневый квартира', 'двухэтажный квартира', 'квартира двухэтажный', 'двухуровневый пентхаус', 'пентхаус двухуровневый', 'первый уровень квартира', 'квартира трехуровневый', 'трехуровневый квартира', 'двухуровневый однокомнатный квартира', 'двухуровневый к квартира', 'уровневый к квартира', 'двухуровневый комнатный квартира', 'расположиться на два уровень', 'двухуровневый трёшка', 'квартира в два уровень', 'двухуровневый х комнатный квартира', 'уровневый х комнатный квартира', 'продаваться двухуровневый', 'продать двухуровневый', 'уровневый пентхаус', 'м в два уровень', 'двухуровневый планировка', 'этажный квартира', 'пентхаус в два уровень', 'продаваться х уровневый', 'квартира находиться в уровень', 'квартира в уровень', 'состоять из два уровень', 'уровнять пентхаус', 'квартира х уровневый', 'двухъярусный квартира', 'доступ в квартира с и с этаж', 'дв

In [52]:
def is_trash(text):
    for word in not_trash_word_lem:  
        if word in text:
            return False
    for word in trash_word_lem:
        if word in text:
            return True
    return False

In [53]:
df['Map_floor'] = df['Текст_lemm_nltk'].apply(is_trash)
map_floor = df[df['Map_floor'] == True]
map_floor

,ID,Мусор,dup,outlier,out_PCA,Цена,Цена_перв,LN(Цена),BC(Цена),Цена_перв.1,...,ПОИСК,Текст,Ремонт,Общая площадь,Год постройки,Этаж/Этажность,Ссылка,Текст_lemm_nltk,Map_floor,Map_terrace
295,296_avito,0,0,1,1.0,23450000,23 450 000 ₽,16.970381,4.962218,NaN,...,0.0,Внесён Задаток. Хотите жить возле Казанского к...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,внести задаток хотеть жить возле казанский кре...,True,False
306,307_avito,0,0,0,1.0,10899000,10 899 000 ₽,16.204182,4.931531,NaN,...,0.0,Идеальный вариант для Инвестирования! Самый пр...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,идеальный вариант для инвестирование самый пре...,True,False
585,586_avito,0,0,0,1.0,27894757,27 894 757 ₽,17.143949,4.968558,NaN,...,0.0,Объект №98917. Хочется жить в центре!? Предлаг...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,объект хотеться жить в центр предлагать ваш вн...,True,False
755,756_avito,0,0,0,0.0,18500000,18 500 000 ₽,16.733281,4.953205,NaN,...,0.0,Объект №97507. Отличный вариант для большой се...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,объект отличный вариант для большой семья прод...,True,False
871,872_avito,0,0,0,0.0,22500000,22 500 000 ₽,16.929026,4.960675,NaN,...,0.0,Арт. 58071162 Предлагаем Вашему вниманию трехк...,евро,NaN,NaN,NaN,https://www.avito.ru/kazan/kvartiry/4-k._kvart...,арт предлагать ваш внимание трехкомнатный двух...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11212,2816_этажи,0,1,0,NaN,15500000,15 500 000,16.556351,4.946203,NaN,...,0.0,"ПРОДАЕТСЯ!!!! Потрясающая квартира для тех, кт...",ремонт по дизайн - проекту,137 м²,1995,1 из 5,https://kazan.etagi.com/realty/8940558/,продаваться потрясать квартира для тот кто люб...,True,False
11224,2828_этажи,0,0,1,NaN,90000000,90 000 000,18.315320,5.006191,NaN,...,0.0,ПРОДАЕТСЯ САМАЯ БОЛЬШАЯ КВАРТИРА В КАЗАНИ!\n\n...,современный ремонт,316.9 м²,2003,4 из 4,https://kazan.etagi.com/realty/8868700/,продаваться самый больший квартира в казань не...,True,True
11235,2839_этажи,0,0,0,0.0,25600000,25 600 000,17.058103,4.965449,NaN,...,0.0,Продаётся эксклюзивная двухуровневая квартира ...,современный ремонт,152.2 м²,2012,6 из 10,https://kazan.etagi.com/realty/8831750/,продаваться эксклюзивный двухуровневый квартир...,True,False
11453,3057_этажи,0,0,0,0.0,19900000,19 900 000,16.806230,4.956022,20000000.0,...,0.0,Продается двухуровневая квартира площадью 200 ...,современный ремонт,200 м²,2000,2 из 9,https://kazan.etagi.com/realty/5236720/,продаваться двухуровневый квартира площадь кв ...,True,False


In [47]:
#Сохраняем в Excel
writer = pd.ExcelWriter('floor_kzn.xlsx')
map_floor.to_excel(writer, 'map_floor')
writer._save()

C:\Users\Silm\AppData\Local\Temp\ipykernel_9192\139851924.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  map_floor.to_excel(writer, 'map_floor')
